In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In [3]:
case class Message(source: Long, destination: Long, value: Double)

defined class Message


In [52]:
val fileName = "../data/input.txt"
var lines = sc.textFile(fileName).collect()
var variablesNumber: Long = 0
var factorsNumber: Long = 0
variablesNumber = lines(0).split(" ")(0).toLong
factorsNumber = lines(0).split(" ")(1).toLong
var tempNodes: Array[(Long, (Int, Int, Array[Double]))] = new Array[(Long, (Int, Int,  Array[Double]))]((variablesNumber+factorsNumber).toInt) // (id, (type, leaf, distribution))
for (a <- 0 to (variablesNumber+factorsNumber - 1).toInt){
    tempNodes(a) = (a.toLong, (0,0, null))
}
var tempEdges: Array[Edge[Long]] = new Array[Edge[Long]](((variablesNumber+factorsNumber-1)*2).toInt)
var cont = 0
for (a <- 1 to lines.size-1){
    if(a%2 == 1){
        val factorNode = variablesNumber + ((a-1)/2).toLong
        var pos = 0
        for (variableNode <- lines(a).split(" ")){
            tempEdges(cont) = Edge(variableNode.toLong, factorNode, 0)
            cont = cont + 1
            tempEdges(cont) = Edge(factorNode, variableNode.toLong, pos)
            cont = cont + 1
            pos = pos + 1
        } 
    }else{
        val distribution:  Array[Double] = lines(a).split(" ").map(_.toDouble)
        val factorNode = (variablesNumber + ((a-1)/2).toLong).toInt
        tempNodes(factorNode) = (factorNode, (1, 0, distribution))
    }
}

fileName = ../data/input.txt
lines = Array(5 5, 0 1, 0.25 0.25 0.25 0.25, 1 2 3, 0.16706443914081145 0.01909307875894988 0.18138424821002386 0.1431980906921241 0.02386634844868735 0.15513126491646778 0.07159904534606205 0.2386634844868735, 2, 0.4 0.6, 3 4, 0.20 0.30 0.40 0.10, 3, 0.25 0.75)
variablesNumber = 5
factorsNumber = 5
variablesNumber = 5
factorsNumber = 5
tempNodes = Array((0,(0,0,null)), (1,(0,0,null)), (2,(0,0,null)), (3,(0,0,null)), (4,(0,0,null)), (5,(1,0,Array(0.25, 0.25, 0.25, 0.25))), (6,(1,0,Array(0.16706443914081145, 0.01909307875894988, 0.18138424821002386, 0.1431980906921241, 0.02386634844868735, 0.15513126491646778, 0.07159904534606205, 0.2386634844868735))), (7,(1,0,Array(0.4,...


Array((0,(0,0,null)), (1,(0,0,null)), (2,(0,0,null)), (3,(0,0,null)), (4,(0,0,null)), (5,(1,0,Array(0.25, 0.25, 0.25, 0.25))), (6,(1,0,Array(0.16706443914081145, 0.01909307875894988, 0.18138424821002386, 0.1431980906921241, 0.02386634844868735, 0.15513126491646778, 0.07159904534606205, 0.2386634844868735))), (7,(1,0,Array(0.4,...

In [45]:
val nodes: RDD[(VertexId, (Int, Int,  Array[Double]))] = sc.parallelize(tempNodes)
val edges: RDD[Edge[Long]] = sc.parallelize(tempEdges)

nodes = ParallelCollectionRDD[112] at parallelize at <console>:37
edges = ParallelCollectionRDD[113] at parallelize at <console>:38


ParallelCollectionRDD[113] at parallelize at <console>:38

In [39]:
val dimensions = (2,2)
val matrix = Array.ofDim[Int](dimensions)

Name: Unknown Error
Message: <console>:35: error: overloaded method value ofDim with alternatives:
  (n1: Int,n2: Int,n3: Int,n4: Int,n5: Int)(implicit evidence$7: scala.reflect.ClassTag[Int])Array[Array[Array[Array[Array[Int]]]]] <and>
  (n1: Int,n2: Int,n3: Int,n4: Int)(implicit evidence$6: scala.reflect.ClassTag[Int])Array[Array[Array[Array[Int]]]] <and>
  (n1: Int,n2: Int,n3: Int)(implicit evidence$5: scala.reflect.ClassTag[Int])Array[Array[Array[Int]]] <and>
  (n1: Int,n2: Int)(implicit evidence$4: scala.reflect.ClassTag[Int])Array[Array[Int]] <and>
  (n1: Int)(implicit evidence$3: scala.reflect.ClassTag[Int])Array[Int]
 cannot be applied to ((Int, Int))
       val matrix = Array.ofDim[Int](dimensions)
                               ^

StackTrace: 

In [2]:
val initialMsg = -9999
// (vertexID, (new vertex value, old vertex value))
val vertices: RDD[(VertexId, (Int, Int))] = sc.parallelize(Array((1L, (1, -1)),
  (2L, (2, -1)), (3L, (3, -1)), (6L, (6, -1))))
val relationships: RDD[Edge[Boolean]] = sc.parallelize(Array(Edge(1L, 2L, true),
  Edge(2L, 1L, true), Edge(2L, 6L, true), Edge(3L, 6L, true), Edge(6L, 1L, true),
  Edge(6L, 3L, true)))
val graph = Graph(vertices, relationships)

def mergeMsg(msg1: Int, msg2: Int): Int = math.max(msg1, msg2)

// Apply: the function for receiving messages
def vprog(vertexId: VertexId, value: (Int, Int), message: Int): (Int, Int) = {
  if (message == initialMsg) // superstep 0
    value
  else // superstep > 0
    (math.max(message, value._1), value._1) // return (newValue, oldValue)
}

// Scatter: the function for computing messages
def sendMsg(triplet: EdgeTriplet[(Int, Int), Boolean]): Iterator[(VertexId, Int)] = {
  val sourceVertex = triplet.srcAttr
  if (sourceVertex._1 == sourceVertex._2) // newValue == oldValue for source vertex?
    Iterator.empty // do nothing
  else
  // propogate new (updated) value to the destination vertex
    Iterator((triplet.dstId, sourceVertex._1))
}

val minGraph = graph.pregel(initialMsg,
  Int.MaxValue,
  EdgeDirection.Out)(
  vprog, // apply
  sendMsg, // scatter
  mergeMsg) // gather
minGraph.vertices.collect.foreach {
  case (vertexId, (value, original_value)) => println(value)
}

6
6
6
6


initialMsg = -9999
vertices = ParallelCollectionRDD[0] at parallelize at <console>:36
relationships = ParallelCollectionRDD[1] at parallelize at <console>:38
graph = org.apache.spark.graphx.impl.GraphImpl@674c5d8a
minGraph = org.apache.spark.graphx.impl.GraphImpl@2e9b2086


mergeMsg: (msg1: Int, msg2: Int)Int
vprog: (vertexId: org.apache.spark.graphx.VertexId, value: (Int, Int), message: Int)(Int, Int)
sendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[(Int, Int),Boolean])Iterator[(org.apache.spark.graphx.VertexId, Int)]


org.apache.spark.graphx.impl.GraphImpl@2e9b2086

2
0.4 0.6
3 4
0.20 0.30 0.40 0.10
3
0.25 0.75
5 5
0 1
0.25 0.25 0.25 0.25
1 2 3
0.16706443914081145 0.01909307875894988 0.18138424821002386 0.1431980906921241 0.02386634844868735 0.15513126491646778 0.07159904534606205 0.2386634844868735
